# generate data

In [39]:
import scipy
import numpy as np
import math

In [25]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [4]:
theta_S = [3., 4., 5., 6.]
theta_T = [3., 4., 5., 3.]
p = len(theta_T)

In [31]:
n_source = 10000
n_target = 100

In [32]:
X_S = np.random.normal(0, 1, size = (n_source, p))
X_T = np.random.normal(0, 1, size = (n_target, p))

In [33]:
g_S = X_S @ theta_S
g_T = X_T @ theta_T

In [34]:
y_S = [np.random.binomial(1, sigmoid(g_Si), 1)[0] for g_Si in g_S]
y_T = [np.random.binomial(1, sigmoid(g_Ti), 1)[0] for g_Ti in g_T]

# pytorch

In [37]:
from torch import nn
from torch.nn import functional as F


In [47]:
class BinaryClassification(nn.Module):
    def __init__(self, input_shape):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(input_shape, 10) 
        self.layer_out = nn.Linear(10, 1) 

        
    def forward(self, inputs):
        x = torch.relu(self.layer_1(inputs))
        x = torch.relu(self.layer_out(x))
        
        return x

In [93]:
X_S_train = torch.tensor(X_S[0:8000, :].astype(np.float32), dtype=torch.float32)
X_S_test = torch.tensor(X_S[8001:-1, :].astype(np.float32), dtype=torch.float32)
y_S_train = torch.tensor(y_S[0:8000], dtype=torch.float32)
y_S_test = torch.tensor(y_S[8001:-1], dtype=torch.float32)


X_T_train = torch.tensor(X_T[0:8000, :].astype(np.float32), dtype=torch.float32)
X_T_test = torch.tensor(X_T[8001:-1, :].astype(np.float32), dtype=torch.float32)
y_T_train = torch.tensor(y_T[0:8000], dtype=torch.float32)
y_T_test = torch.tensor(y_T[8001:-1], dtype=torch.float32)

In [98]:
learning_rate = 0.01
epochs = 700
# Model , Optimizer, Loss
source_model = BinaryClassification(input_shape=X_S_train.shape[1])
optimizer = torch.optim.SGD(source_model.parameters(),lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()

In [99]:
source_model(X_S_train)

tensor([[0.1185],
        [0.1508],
        [0.1480],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]], grad_fn=<ReluBackward0>)

In [100]:
losses = []
accur = []
for i in range(epochs):

    #calculate output
    output = source_model(X_S_train)

    #calculate loss
    loss = loss_fn(output,y_S_train.reshape(-1,1))

    #accuracy
#     predicted = source_model(torch.tensor(x,dtype=torch.float32))
#     acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.append(loss)
#     accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy :".format(i,loss))


epoch 0	loss : 0.7090245485305786	 accuracy :
epoch 1	loss : 0.7088062167167664	 accuracy :
epoch 2	loss : 0.7085909843444824	 accuracy :
epoch 3	loss : 0.7083795070648193	 accuracy :
epoch 4	loss : 0.7081717252731323	 accuracy :
epoch 5	loss : 0.7079682350158691	 accuracy :
epoch 6	loss : 0.7077677249908447	 accuracy :
epoch 7	loss : 0.7075690627098083	 accuracy :
epoch 8	loss : 0.7073742747306824	 accuracy :
epoch 9	loss : 0.7071831226348877	 accuracy :
epoch 10	loss : 0.7069954872131348	 accuracy :
epoch 11	loss : 0.706810474395752	 accuracy :
epoch 12	loss : 0.7066283226013184	 accuracy :
epoch 13	loss : 0.7064484357833862	 accuracy :
epoch 14	loss : 0.7062703967094421	 accuracy :
epoch 15	loss : 0.7060937285423279	 accuracy :
epoch 16	loss : 0.7059194445610046	 accuracy :
epoch 17	loss : 0.705747663974762	 accuracy :
epoch 18	loss : 0.705578088760376	 accuracy :
epoch 19	loss : 0.705410897731781	 accuracy :
epoch 20	loss : 0.7052456736564636	 accuracy :
epoch 21	loss : 0.705082714

epoch 226	loss : 0.694100558757782	 accuracy :
epoch 227	loss : 0.6940891742706299	 accuracy :
epoch 228	loss : 0.6940778493881226	 accuracy :
epoch 229	loss : 0.6940667033195496	 accuracy :
epoch 230	loss : 0.6940559148788452	 accuracy :
epoch 231	loss : 0.6940451860427856	 accuracy :
epoch 232	loss : 0.6940346360206604	 accuracy :
epoch 233	loss : 0.6940242648124695	 accuracy :
epoch 234	loss : 0.6940140128135681	 accuracy :
epoch 235	loss : 0.6940039992332458	 accuracy :
epoch 236	loss : 0.6939938068389893	 accuracy :
epoch 237	loss : 0.693983793258667	 accuracy :
epoch 238	loss : 0.6939736008644104	 accuracy :
epoch 239	loss : 0.6939635276794434	 accuracy :
epoch 240	loss : 0.6939535737037659	 accuracy :
epoch 241	loss : 0.693943977355957	 accuracy :
epoch 242	loss : 0.6939343214035034	 accuracy :
epoch 243	loss : 0.693924605846405	 accuracy :
epoch 244	loss : 0.6939152479171753	 accuracy :
epoch 245	loss : 0.6939060091972351	 accuracy :
epoch 246	loss : 0.693897008895874	 accuracy

epoch 477	loss : 0.693146824836731	 accuracy :
epoch 478	loss : 0.693145751953125	 accuracy :
epoch 479	loss : 0.6931447386741638	 accuracy :
epoch 480	loss : 0.6931436061859131	 accuracy :
epoch 481	loss : 0.6931426525115967	 accuracy :
epoch 482	loss : 0.6931414604187012	 accuracy :
epoch 483	loss : 0.6931405067443848	 accuracy :
epoch 484	loss : 0.6931395530700684	 accuracy :
epoch 485	loss : 0.6931384205818176	 accuracy :
epoch 486	loss : 0.6931374669075012	 accuracy :
epoch 487	loss : 0.69313645362854	 accuracy :
epoch 488	loss : 0.6931354403495789	 accuracy :
epoch 489	loss : 0.6931344866752625	 accuracy :
epoch 490	loss : 0.6931334137916565	 accuracy :
epoch 491	loss : 0.6931325197219849	 accuracy :
epoch 492	loss : 0.6931314468383789	 accuracy :
epoch 493	loss : 0.6931304931640625	 accuracy :
epoch 494	loss : 0.6931294202804565	 accuracy :
epoch 495	loss : 0.6931284666061401	 accuracy :
epoch 496	loss : 0.6931275129318237	 accuracy :
epoch 497	loss : 0.6931264996528625	 accurac

epoch 695	loss : 0.6929501891136169	 accuracy :
epoch 696	loss : 0.6929492354393005	 accuracy :
epoch 697	loss : 0.6929482221603394	 accuracy :
epoch 698	loss : 0.6929473280906677	 accuracy :
epoch 699	loss : 0.6929464936256409	 accuracy :
